In [52]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GATConv, GINConv, MLP
import torch_geometric.transforms as T
from sklearn.model_selection import train_test_split

import os 

cwd = os.getcwd()
cwd = os.path.join(cwd, 'data')

dataset = Planetoid(root=cwd, name='Cora')
dataset1 = Planetoid(root=cwd, name='CiteSeer')
dataset2 = Planetoid(root = cwd, name= 'PubMed')

datasets = [dataset, dataset1, dataset2]


In [53]:

# class GCN(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super().__init__()
#         # Pre-process normalization to avoid CPU communication/graph breaks:
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.conv2(x, edge_index)
#         return x

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x

    
class GIN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GIN, self).__init__()
        self.convs = torch.nn.ModuleList()
        for _ in range(2):
            mlp = MLP([num_features, hidden_channels, hidden_channels])
            self.convs.append(GINConv(nn=mlp, train_eps=False))
            num_features = hidden_channels

        self.mlp = MLP([hidden_channels, hidden_channels, num_classes], norm=None, dropout=0.5)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index).relu()
        return self.mlp(x) 

    

class GAT(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, hidden_channels, heads=heads, dropout= 0.6)
        self.conv2 = GATConv(hidden_channels * heads, num_classes, heads=heads, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [55]:


# Training and evaluation functions
def train(model, optimizer, data, train_mask):
    model.train()
    optimizer.zero_grad()
    if model == GCN:
        out = model(data.x, data.edge_index, data.edge_weight)
    else: 
        out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, mask):
    model.eval()
    logits = model(data.x, data.edge_index)
    loss = F.cross_entropy(logits[mask], data.y[mask])
    pred = logits[mask].max(1)[1]
    correct = pred.eq(data.y[mask]).sum().item()
    accuracy = correct / mask.sum().item()
    return accuracy, loss.item()

def split_indices(data, train_ratio=0.1, val_ratio=0.1, test_ratio=0.8):
    indices = np.arange(data.num_nodes)
    train_size = int(train_ratio * data.num_nodes)
    val_size = int(val_ratio * data.num_nodes)
    test_size = int(test_ratio * data.num_nodes)
    
    train_indices, temp_indices = sklearn_train_test_split(indices, train_size=train_size, random_state=42)
    val_indices, test_indices = sklearn_train_test_split(temp_indices, test_size=test_size, random_state=42)
    
    train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    
    train_mask[train_indices] = True
    val_mask[val_indices] = True
    test_mask[test_indices] = True
    
    return train_mask, val_mask, test_mask


In [57]:

def run_experiment(model_class, num_features, num_classes, data, num_runs=5, epochs=300):
    for dataset in datasets:
        accuracies = []
        for _ in range(num_runs):
            print(f"Run {_ + 1}/{num_runs} on dataset {dataset.name}")
            train_mask, val_mask, test_mask = split_indices(data)
            
            if model_class == GCN:
                model = model_class(num_features, hidden_channels=256, num_classes=num_classes).to(device)
            model = model_class(num_features, hidden_channels=256, num_classes=num_classes).to(device)
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            
            best_val_acc = 0.0
            best_model_state = None
            
            for epoch in range(epochs):
                train_loss = train(model, optimizer, data, train_mask)
                val_acc, val_loss = evaluate(model, data, val_mask)
                test_acc, test_loss = evaluate(model, data, test_mask)

                if epoch % 200 == 0:
                
                    print(f"Epoch: {epoch + 1}/{epochs} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")
                
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_model_state = model.state_dict()
            
            if best_model_state:
                model.load_state_dict(best_model_state)
            
            test_acc, _ = evaluate(model, data, test_mask)
            accuracies.append(test_acc)
        
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        
        print(f"\nMean Accuracy over {num_runs} runs: {mean_acc:.4f}, Std Deviation: {std_acc:.4f}")
        print()

In [62]:


def main():
    print(f'Dataset: {dataset.name}')
    data = dataset[0]  # Assuming dataset is a tuple where [0] is the data object

    global device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    num_features = data.num_features
    num_classes = dataset.num_classes

    for model_class in [GCN, GAT, GIN]:
        print('')
        print(f'{model_class}')
        run_experiment(model_class, num_features, num_classes, data)

if __name__ == '__main__':
    main()

Dataset: Cora

<class '__main__.GCN'>
Run 1/5 on dataset Cora
Epoch: 1/300 | Train Loss: 1.9518 | Val Acc: 0.4926 | Test Acc: 0.5125
Epoch: 201/300 | Train Loss: 0.0000 | Val Acc: 0.7941 | Test Acc: 0.8287
Run 2/5 on dataset Cora
Epoch: 1/300 | Train Loss: 1.9632 | Val Acc: 0.4338 | Test Acc: 0.4534
Epoch: 201/300 | Train Loss: 0.0000 | Val Acc: 0.7941 | Test Acc: 0.8269
Run 3/5 on dataset Cora
Epoch: 1/300 | Train Loss: 1.9439 | Val Acc: 0.3603 | Test Acc: 0.3869
Epoch: 201/300 | Train Loss: 0.0000 | Val Acc: 0.7941 | Test Acc: 0.8319
Run 4/5 on dataset Cora
Epoch: 1/300 | Train Loss: 1.9501 | Val Acc: 0.4228 | Test Acc: 0.4460
Epoch: 201/300 | Train Loss: 0.0000 | Val Acc: 0.7904 | Test Acc: 0.8278
Run 5/5 on dataset Cora
Epoch: 1/300 | Train Loss: 1.9504 | Val Acc: 0.3897 | Test Acc: 0.4021
Epoch: 201/300 | Train Loss: 0.0000 | Val Acc: 0.7978 | Test Acc: 0.8283

Mean Accuracy over 5 runs: 0.8279, Std Deviation: 0.0014

Run 1/5 on dataset CiteSeer
Epoch: 1/300 | Train Loss: 1.9549 |

KeyboardInterrupt: 